# Medium scraper

Script to scrape Medium articles based on tag and date range.  

## Initialization

In [1]:
import re
from datetime import date,timedelta
import time
import csv

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import random

import nltk
from nltk.tokenize import word_tokenize

from selenium import webdriver
from selenium.webdriver.common.by import By


In [2]:
#PARAMETERS 

#Tags to scrape from
TAGS = [
        'data-science'
]

#Date range to scrape data
DATE_FROM = date(2016,1,1) 
DATE_TO = date(2023,12,31)

#time interval for scraping
INTERVAL = 3

# Maximum number of articles to scrape per day
MAX_PER_DAY = 10 
MAX_RETRIES = 3

#email address used to sign in to medium (to scrape member-only articles
MEDIUM_EMAIL = 'hiroakiroa@gmail.com'


## Scrape article list

In [7]:
#Initialize
curr_date = DATE_FROM
delta = timedelta(days=1)
df = pd.DataFrame(columns=["url","tag","date"])


with open("article_list.csv", "w") as file:
    file.write("url,tag,date\n")
    
    for tag in TAGS:
        while curr_date <= DATE_TO:
            retry = 0
            response = None
            
            #Scrape list of articles through the tag archive site in Medium
            while (retry < MAX_RETRIES and (response == None or response.status_code != 200)):
                response = requests.get('https://medium.com/tag/' + tag + "/archive/" +curr_date.strftime("%Y/%m/%d"))
                retry += 1
                time.sleep(INTERVAL)
            if response.status_code == 200:
                # Parse the HTML content of the page using BeautifulSoup
                soup = BeautifulSoup(response.content, "html.parser")

                anchor_tags = soup.find_all('a')
                for anchor in anchor_tags:
                    
                    # As of Mar 2024 it seems the article titles (which are hyperlinks) are set as h3. Using this as the tag to find the article urls
                    if (anchor.find('h3') is not None):
                        href = anchor.get('href')
                        href = re.findall("[^\?]+", href)[0]
                        file.write(f'"{href}","{tag}","{curr_date.strftime("%Y-%m-%d")}"\n')
            else:
                print("Failed to retrieve the webpage.")
            curr_date += delta
            print(curr_date, end=" ")
            
    file.close()


2016-01-02 2016-01-03 2016-01-04 2016-01-05 2016-01-06 2016-01-07 2016-01-08 2016-01-09 2016-01-10 2016-01-11 2016-01-12 2016-01-13 2016-01-14 2016-01-15 2016-01-16 2016-01-17 2016-01-18 2016-01-19 2016-01-20 2016-01-21 2016-01-22 2016-01-23 2016-01-24 2016-01-25 2016-01-26 2016-01-27 2016-01-28 2016-01-29 2016-01-30 2016-01-31 2016-02-01 2016-02-02 2016-02-03 2016-02-04 2016-02-05 2016-02-06 2016-02-07 2016-02-08 2016-02-09 2016-02-10 2016-02-11 2016-02-12 2016-02-13 2016-02-14 2016-02-15 2016-02-16 2016-02-17 2016-02-18 2016-02-19 2016-02-20 2016-02-21 2016-02-22 2016-02-23 2016-02-24 2016-02-25 2016-02-26 2016-02-27 2016-02-28 2016-02-29 2016-03-01 2016-03-02 2016-03-03 2016-03-04 2016-03-05 2016-03-06 2016-03-07 2016-03-08 2016-03-09 2016-03-10 2016-03-11 2016-03-12 2016-03-13 2016-03-14 2016-03-15 2016-03-16 2016-03-17 2016-03-18 2016-03-19 2016-03-20 2016-03-21 2016-03-22 2016-03-23 2016-03-24 2016-03-25 2016-03-26 2016-03-27 2016-03-28 2016-03-29 2016-03-30 2016-03-31 2016-04-01

## Scrape articles

### Initialization for scraping via colab

In [ ]:
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb

wget -N https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/118.0.5993.70/linux64/chromedriver-linux64.zip -P /tmp/
unzip -o /tmp/chromedriver-linux64.zip -d /tmp/
chmod +x /tmp/chromedriver-linux64/chromedriver
mv /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver_autoinstaller



In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
import chromedriver_autoinstaller

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # this is must
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_autoinstaller.install()

driver = webdriver.Chrome(options=chrome_options)

### Medium sign in
This step is required to scrape member articles. Note that the script uses selinium and browser interactions, which makes it more vulnerable to Medium site changes. 

In [ ]:
driver.get("https://medium.com")
driver.implicitly_wait(3)
element = driver.find_element(By.XPATH,"//a[text()='Sign in']")
element.click()
driver.implicitly_wait(0.5)
sign_in_with_email = driver.find_element(By.XPATH,'//div[text()="Sign in with email"]')

# Perform actions on the element (e.g., click)
sign_in_with_email.click()
driver.implicitly_wait(0.5)
email = driver.find_element(By.XPATH,"//input[@aria-label='email']")
email.click()
driver.implicitly_wait(0.5)

email.send_keys(MEDIUM_EMAIL)
driver.implicitly_wait(1)
cont = driver.find_element(By.XPATH,'//button[text()="Continue"]')
cont.click()

The script above should send a login verification email to your email address. Once you receive the email address, copy the URL and paste to the _signin_url_ field below

In [ ]:
## 
signin_url = ''
driver.get(signin_url)


### Scrape articles

In [ ]:
df = pd.read_csv('article_list.csv')
# If you don't want to scrape all the articles in the article_list, filter the days field below. 
# Note that the date field is stored as a string in the format of  YYYY-mm-DD
days = df[df['date']]['date'].unique()

article_df = pd.DataFrame(columns=["url","tag","date","content"])


with open("articles.csv", "w",encoding="utf-8") as file:
    writer = csv.writer(file, dialect='excel')
    writer.writerow(['url','tag','date','content'])
    for day in days:
        print(day, end=" ")
        # If number of articles is more than the MAX_ARTICLES_PER_DAY, sample
        if df.loc[df['date']==day,:].shape[0] > MAX_ARTICLES_PER_DAY:
          sampled_urls = df.loc[df['date']==day,:].sample(n=MAX_ARTICLES_PER_DAY,replace=False,random_state=0)
        else:
          sampled_urls = df.loc[df['date']==day,:]
        for index, row in sampled_urls.iterrows():
            url= row['url']
            tag= row['tag']
            article_date = row['date']
            content = ""
            retry = 0
            response = None
            try:
                driver.get(url)
                time.sleep(INTERVAL)

            except Exception as e:
                    print("An error occurred:", e)
                    continue
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            article = soup.find('article')
            if article is not None:
                
                # The script assumes all the texts with <p> tag are the article's main text. This can definitely be improved. 
                for para in article.find_all("p"): 
                    content += para.get_text() + chr(10)
                writer.writerow([url,tag,article_date,content])
file.close()
            
    